# DATA ANALYSIS Smoke Metric and Asthma Modeling

## Sid Gurajala
## Last Updated: 11/18/2024

### Read in Files and Import Libraries

In [79]:
import pandas as pd
import os 
import numpy as np
from scipy.stats import spearmanr, pearsonr
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels.api as sm

cwd = os.getcwd()
data_dir = os.path.join(cwd, "../../data/")
image_dir = os.path.join(cwd, "../../img/")
#join file paths 
gbd_death_metric_file_path = os.path.join(data_dir, "final/gbd_aggregated_death_metric_per_year.csv")
gbd_incidence_metric_file_path = os.path.join(data_dir, "final/gbd_aggregated_incidence_metric_per_year.csv")
ncei_file_path = os.path.join(data_dir, "final/ncei_cleaned_final.csv")
wildfire_fire_path = os.path.join(data_dir, "final/USGS_wildfire_dearborn_filtered.csv")
aqi_file_path = os.path.join(data_dir, "final/AQI_Dearborn_Michigan.csv")

#read in file
gbd_death_df = pd.read_csv(gbd_death_metric_file_path)
gbd_incidence_df = pd.read_csv(gbd_incidence_metric_file_path)
ncei_df = pd.read_csv(ncei_file_path)
wildfire_df = pd.read_csv(wildfire_fire_path)
aqi_df = pd.read_csv(aqi_file_path)

### Creating The Smoke Metric

 The metric I chose to create therefore takes the following form. It is grounded in principles from the inverse square law and integrates the climate data as ratios of temperature precipitation and drought severity index. We assigned temperature, drought severity index, and precipitation values to a given fire by unifying across the year. 

$$SmokeMetric_{fire} = f(acreage_{fire}, distance_{fire}, temperature_{fire}, dsi_{fire}, precipitation_{fire})$$
$$f(acreage_{fire}, distance_{fire}, temperature_{fire}, dsi_{fire}, precipitation_{fire}) = \frac{acreage_{fire}}{{distance}^2_{fire}} * (\frac{temperature_{fire}}{precipitation_{fire} * dsi_{fire}})^2$$
$$SmokeMetric_{year} = \sum_{x=0}^{n} f(acreage_{x}, distance_{x})$$ 

We rescaled the average drought severity index (DSI) to a normalized range between 0 and 1, adding a small offset (0.0001) to prevent division by zero or zero values.

In [41]:
# Rescale the average Drought Severity Index (DSI) to a range between 0 and 1, with a small offset added for numerical stability
ncei_df['rescaled_avg_DSI'] = (
    (ncei_df.avg_Drought_Severity_Index - min(ncei_df.avg_Drought_Severity_Index)) /
    (max(ncei_df.avg_Drought_Severity_Index) - min(ncei_df.avg_Drought_Severity_Index)) + 0.0001
)

We merged wildfire and climate datasets, filtered for wildfires within 500 km of Dearborn, and calculated two smoke impact metrics: one using acreage, distance, and climate variables, and computed the previous metric using only acreage and distance. Then, we aggregated the metrics and total wildfire acreage by year for further analysis.

In [ ]:
# Merge wildfire and climate data, excluding 'avg_Drought_Severity_Index' from the climate dataset
wildfire_merged_df = wildfire_df.merge(
    ncei_df.drop("avg_Drought_Severity_Index", axis=1), how="inner", on="Year"
)

# Filter for wildfires that occurred within 500 km of Dearborn
wildfire_merged_df = wildfire_merged_df[wildfire_merged_df.Distance_to_Dearborn <= 500]

# Calculate a refined smoke impact metric considering acreage, distance, temperature, precipitation, and drought index
wildfire_merged_df['Smoke_Impact_Metric'] = (
    (wildfire_merged_df.Acreage / wildfire_merged_df.Distance_to_Dearborn ** 2) *
    ((wildfire_merged_df.avg_Temperature /
      (wildfire_merged_df.avg_Precipitation * wildfire_merged_df.rescaled_avg_DSI)) ** 2)
)

# Calculate a simpler smoke impact metric using only acreage and distance
wildfire_merged_df['Smoke_Impact_Metric_Old'] = (
    wildfire_merged_df.Acreage / (wildfire_merged_df.Distance_to_Dearborn ** 2)
)

# Compute the yearly average of both smoke impact metrics
wildfire_aggregated_yearly = wildfire_merged_df.groupby(
    "Year", as_index=False
)[['Smoke_Impact_Metric', 'Smoke_Impact_Metric_Old']].mean()

Next, we calculated the yearly average Air Quality Index (AQI) from the AQI dataset and renamed the resulting columns for clarity.

In [ ]:
# Group the AQI dataset by year and calculate the average AQI for each year
aqi_df_yearly = aqi_df.groupby("year", as_index=False)['aqi'].mean()

# Rename the columns to merge 
aqi_df_yearly.columns = ["Year", "avg_AQI"]

Next, we merged the smoke impact metrics with AQI data by year, capped the new metric at 5, and calculated its rolling and exponential moving averages. Then, we filtered the data for years after 1984 and computed Spearman and Pearson correlation coefficients to assess the relationship between AQI and both old and new smoke impact metrics.

In [44]:
# Merge yearly smoke impact metrics with AQI data on the 'Year' column
df = wildfire_aggregated_yearly.merge(aqi_df_yearly, how="inner", on="Year")

# Cap the new smoke impact metric at a maximum value of 5
df.Smoke_Impact_Metric[df.Smoke_Impact_Metric > 5] = 5

# Calculate a 5-year rolling average of the smoke impact metric
df['Rolling_Avg'] = df['Smoke_Impact_Metric'].rolling(window=5).mean()

# Calculate a 5-year exponential moving average (EMA) of the smoke impact metric
df['EMA'] = df['Smoke_Impact_Metric'].ewm(span=5).mean()

# Filter the data to include only years from 1984 onwards
df = df[df.Year >= 1984]

# Compute and print Spearman and Pearson correlation coefficients for new metric
refined_result = spearmanr(df['avg_AQI'], df['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with AQI: SpearmanR = " f'{refined_result.statistic}')
refined_result = pearsonr(df['avg_AQI'], df['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with AQI: PearsonR = " f'{refined_result.statistic}')

# Compute and print Spearman and Pearson correlation coefficients for old metric
refined_result = spearmanr(df['avg_AQI'], df['Smoke_Impact_Metric_Old'])
print("Old Smoke Estimate Correlation with AQI: SpearmanR = " f'{refined_result.statistic}')
refined_result = pearsonr(df['avg_AQI'], df['Smoke_Impact_Metric_Old'])
print("Old Smoke Estimate Correlation with AQI: PearsonR = " f'{refined_result.statistic}')

New Smoke Estimate Correlation with AQI: SpearmanR = 0.5942613399241647
New Smoke Estimate Correlation with AQI: PearsonR = 0.42532544892091145
Old Smoke Estimate Correlation with AQI: SpearmanR = 0.19037458511142719
Old Smoke Estimate Correlation with AQI: PearsonR = -0.03695653144022787


/var/folders/_p/bxhqsmnd5w54bhnc_zblrb9h0000gq/T/ipykernel_51039/3980050645.py:5: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/var/folders/_p/bxhqsmnd5w54bhnc_zblrb9h0000gq/T/ipykernel_51039/3980050645.py:5: SettingWithCopyWarning:


A val

Here, created dual-axis plots using to compare AQI trends with the old and improved smoke impact metrics over time for Dearborn, MI. The first plot shows the old metric alongside AQI, while the second highlights the improved metric, with separate y-axes for AQI and smoke metrics in both cases.

In [45]:
# Create subplots with secondary y-axis enabled
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['avg_AQI'], name="Average AQI", line=dict(color="blue")),
    secondary_y=False, 
)
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Smoke_Impact_Metric_Old'], name="Smoke Impact Metric", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig.update_layout(
    title_text="Average AQI and Previous Smoke Metric Over Time: Dearborn MI",
    xaxis_title="Year",
    yaxis_title="Average AQI",
    yaxis2_title="Smoke Metric",
    template='plotly_white'
)


# Create subplots with secondary y-axis enabled
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig1.add_trace(
    go.Scatter(x=df['Year'], y=df['avg_AQI'], name="Average AQI", line=dict(color="blue")),
    secondary_y=False, 
)
fig1.add_trace(
    go.Scatter(x=df['Year'], y=df['Smoke_Impact_Metric'], name="Smoke Impact Metric", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig1.update_layout(
    title_text="Average AQI and Improved Smoke Metric Over Time: Dearborn MI",
    xaxis_title="Year",
    yaxis_title="Average AQI",
    yaxis2_title="Smoke Metric",
    template='plotly_white'
)

fig.show()

fig1.show()

We created scatter plots to visualize the relationship between AQI and both the old and improved smoke metrics, including linear trendlines. These plots help assess how well each metric correlates with AQI over the years.

In [46]:
fig = px.scatter(
    x=df["Smoke_Impact_Metric"],
    y=df["avg_AQI"],
    trendline = 'ols'
)
fig.update_layout(
    title_text="Improved Smoke Metric vs. Average AQI per Year",
    xaxis_title="Smoke Impact Metric (Improved)",
    yaxis_title="Average AQI",
    template='plotly_white'
)

fig1 = px.scatter(
    x=df["Smoke_Impact_Metric_Old"],
    y=df["avg_AQI"],
    trendline="ols"
)
fig1.update_layout(
    title_text="Previous Smoke Metric vs. Average AQI per Year",
    xaxis_title="Smoke Impact Metric (Previous)",
    yaxis_title="Average AQI",
    template='plotly_white'
)

fig.show()
fig1.show()

### Estimating the Impact on Respiratory Disease Trends

We renamed the columns in the global burden of disease (GBD) datasets for deaths and incidence to make them more descriptive. Next, we dropped the "Measure" column in both datasets to simplify the structure and focus on the relevant data for further analysis.

In [47]:
# Rename columns in the GBD deaths dataset and drop the "Measure" column
gbd_death_df.columns = ["Year", "Measure", "Deaths"]
gbd_death_df = gbd_death_df.drop("Measure", axis=1)

# Rename columns in the GBD incidence dataset and drop the "Measure" column
gbd_incidence_df.columns = ["Year", "Measure", "Incidence"]
gbd_incidence_df = gbd_incidence_df.drop("Measure", axis=1)

Here we merge df with the gbd death dataset and the gbd incidence dataset. The incidence dataset has less years of data than the death dataset so we used a left join rather than inner. 

In [48]:
df = df.merge(gbd_death_df, how = "inner", on = "Year").merge(gbd_incidence_df, how = "left", on = "Year")

We computed the Spearman and Pearson correlation coefficients between various versions of the new smoke estimate (Smoke Impact Metric) and both incidence and death data. 

In [49]:
# Compute Spearman and Pearson correlation coefficients between incidence and smoke impact metric, dropping missing values
refined_result = spearmanr(df.dropna()['Incidence'], df.dropna()['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with Incidence: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df.dropna()['Incidence'], df.dropna()['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with Incidence: PearsonR = " f'{refined_result.statistic}')

# Compute Spearman and Pearson correlation coefficients between deaths and smoke impact metric
refined_result = spearmanr(df['Deaths'], df['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with Deaths: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df['Deaths'], df['Smoke_Impact_Metric'])
print("New Smoke Estimate Correlation with Deaths: PearsonR = " f'{refined_result.statistic}')

New Smoke Estimate Correlation with Incidence: SpearmanR = -0.13231142665851256
New Smoke Estimate Correlation with Incidence: PearsonR = 0.0028764736738194907
New Smoke Estimate Correlation with Deaths: SpearmanR = 0.09651411227020551
New Smoke Estimate Correlation with Deaths: PearsonR = 0.10668761154005073


In [50]:
# Compute Spearman and Pearson correlation coefficients between incidence and rolling average, dropping missing values
refined_result = spearmanr(df.dropna()['Incidence'], df.dropna()['Rolling_Avg'])
print("Rolling Average Correlation with Incidence: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df.dropna()['Incidence'], df.dropna()['Rolling_Avg'])
print("Rolling Average Correlation with Incidence: PearsonR = " f'{refined_result.statistic}')

# Compute Spearman and Pearson correlation coefficients between deaths and rolling average
refined_result = spearmanr(df.dropna()['Deaths'], df.dropna()['Rolling_Avg'])
print("Rolling Average Correlation with Deaths: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df.dropna()['Deaths'], df.dropna()['Rolling_Avg'])
print("Rolling Average Correlation with Deaths: PearsonR = " f'{refined_result.statistic}')

Rolling Average Correlation with Incidence: SpearmanR = 0.1342741935483871
Rolling Average Correlation with Incidence: PearsonR = 0.12334426713049862
Rolling Average Correlation with Deaths: SpearmanR = 0.1774193548387097
Rolling Average Correlation with Deaths: PearsonR = 0.25364005247103244


In [51]:
# Compute Spearman and Pearson correlation coefficients between incidence and EMA, dropping missing values
refined_result = spearmanr(df.dropna()['Incidence'], df.dropna()['EMA'])
print("Weighted Rolling Average Correlation with Incidence: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df.dropna()['Incidence'], df.dropna()['EMA'])
print("Weighted Rolling Average Correlation with Incidence: PearsonR = " f'{refined_result.statistic}')

# Compute Spearman and Pearson correlation coefficients between deaths and EMA
refined_result = spearmanr(df['Deaths'], df['EMA'])
print("Weighted Rolling Average Correlation with Deaths: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df['Deaths'], df['EMA'])
print("Weighted Rolling Average Correlation with Deaths: PearsonR = " f'{refined_result.statistic}')

Weighted Rolling Average Correlation with Incidence: SpearmanR = 0.11169354838709677
Weighted Rolling Average Correlation with Incidence: PearsonR = 0.07514661065341535
Weighted Rolling Average Correlation with Deaths: SpearmanR = 0.4767662399241346
Weighted Rolling Average Correlation with Deaths: PearsonR = 0.4514848496485252


In [52]:
# Compute Spearman and Pearson correlation coefficients between incidence and average AQI, dropping missing values
refined_result = spearmanr(df.dropna()['Incidence'], df.dropna()['avg_AQI'])
print("AQI Correlation with Incidence: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df.dropna()['Incidence'], df.dropna()['avg_AQI'])
print("AQI Correlation with Incidence: PearsonR = " f'{refined_result.statistic}')

# Compute Spearman and Pearson correlation coefficients between deaths and average AQI
refined_result = spearmanr(df['Deaths'], df['avg_AQI'])
print("AQI Correlation with Deaths: SpearmanR = " f'{refined_result.statistic}')

refined_result = pearsonr(df['Deaths'], df['avg_AQI'])
print("AQI Correlation with Deaths: PearsonR = " f'{refined_result.statistic}')

AQI Correlation with Incidence: SpearmanR = -0.49959677419354837
AQI Correlation with Incidence: PearsonR = -0.5539889642965664
AQI Correlation with Deaths: SpearmanR = -0.1254148885727833
AQI Correlation with Deaths: PearsonR = -0.07912133509761055


We created several plots to visualize the relationship between respiratory disease metrics (deaths and incidence) and various smoke impact metrics (including raw, rolling average, and weighted rolling average). These plots use dual y-axes, with the primary y-axis for the disease metrics and the secondary y-axis for the smoke impact metrics. The plots provide insights into how trends in air quality (represented by smoke metrics) correlate with respiratory health outcomes over time in Dearborn, MI.

In [53]:
# Create subplots with secondary y-axis enabled
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Deaths'], name="Deaths", line=dict(color="blue")),
    secondary_y=False, 
)
fig.add_trace(
    go.Scatter(x=df['Year'], y=df['Smoke_Impact_Metric'], name="Smoke Impact Metric", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig.update_layout(
    title_text="Respiratory Disease Deaths and Smoke Impact Metric",
    xaxis_title="Year",
    yaxis_title="Deaths",
    yaxis2_title="Smoke Metric",
    template='plotly_white'
)


# Create subplots with secondary y-axis enabled
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig1.add_trace(
    go.Scatter(x=df['Year'], y=df['Incidence'], name="Incidence", line=dict(color="blue")),
    secondary_y=False, 
)
fig1.add_trace(
    go.Scatter(x=df['Year'], y=df['Smoke_Impact_Metric'], name="Smoke Impact Metric", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig1.update_layout(
    title_text="Respiratory Disease Incidence and Smoke Impact Metric",
    xaxis_title="Year",
    yaxis_title="Incidence",
    yaxis2_title="Smoke Metric",
    template='plotly_white'
)

# Create subplots with secondary y-axis enabled
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig2.add_trace(
    go.Scatter(x=df['Year'], y=df['Incidence'], name="Incidence", line=dict(color="blue")),
    secondary_y=False, 
)
fig2.add_trace(
    go.Scatter(x=df['Year'], y=df['Rolling_Avg'], name="Metric Rolling Average", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig2.update_layout(
    title_text="Respiratory Disease Incidence and Rolling Average Smoke Impact Metric",
    xaxis_title="Year",
    yaxis_title="Incidence",
    yaxis2_title="Rolling Average Smoke Metric",
    template='plotly_white'
)

# Create subplots with secondary y-axis enabled
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig3.add_trace(
    go.Scatter(x=df['Year'], y=df['Deaths'], name="Incidence", line=dict(color="blue")),
    secondary_y=False, 
)
fig3.add_trace(
    go.Scatter(x=df['Year'], y=df['Rolling_Avg'], name="Metric Rolling Average", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig3.update_layout(
    title_text="Respiratory Disease Deaths and Rolling Avg Smoke Impact Metric",
    xaxis_title="Year",
    yaxis_title="Deaths",
    yaxis2_title="Rolling Average Smoke Metric",
    template='plotly_white'
)

# Create subplots with secondary y-axis enabled
fig4 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig4.add_trace(
    go.Scatter(x=df['Year'], y=df['Deaths'], name="Deaths", line=dict(color="blue")),
    secondary_y=False, 
)
fig4.add_trace(
    go.Scatter(x=df['Year'], y=df['EMA'], name="Weighted Rolling Average", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig4.update_layout(
    title_text="Respiratory Disease Deaths and Weighted Rolling Avg Smoke Impact Metric: Dearborn MI",
    xaxis_title="Year",
    yaxis_title="Deaths",
    yaxis2_title="Weighted Rolling Average Smoke Metric",
    template='plotly_white'
)

# Create subplots with secondary y-axis enabled
fig5 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig5.add_trace(
    go.Scatter(x=df['Year'], y=df['Incidence'], name="Incidence", line=dict(color="blue")),
    secondary_y=False, 
)
fig5.add_trace(
    go.Scatter(x=df['Year'], y=df['EMA'], name="Weighted Rolling Average", line=dict(color="red")),
    secondary_y=True,  
)
# Update layout for dual y-axis titles
fig5.update_layout(
    title_text="Respiratory Disease Incidence and Weighted Rolling Avg Smoke Impact Metric: Dearborn MI",
    xaxis_title="Year",
    yaxis_title="Deaths",
    yaxis2_title="Weighted Rolling Average Smoke Metric",
    template='plotly_white'
)


fig.show()

fig1.show()

fig2.show()

fig3.show()

fig4.show()

fig5.show()

### Forecasting the Smoke Metric

We created a time series dataframe by copying the original dataframe, converting the 'Year' column to a datetime object, setting it as the index, and specifying a yearly frequency starting in January. This prepares the data for time series analysis, where each row now represents a year.

In [54]:
#copy the fire dataframe
df_ts = df.copy()
#change year to datetime column 
df_ts["Year"] = pd.to_datetime(df_ts["Year"], format='%Y')
#set index and specify frequency
df_ts = df_ts.set_index("Year")
df_ts = df_ts.asfreq("YS-JAN")

We fit an Exponential Smoothing model to the 'Smoke_Impact_Metric' time series data, including additive components for both trend and seasonality with a seasonal period of 12. The model is then used to forecast the next 30 years.



In [55]:
#fit exponential smoothing model
ts_model = ExponentialSmoothing(
    df_ts['Smoke_Impact_Metric'],
    trend="add",
    seasonal="add",
    seasonal_periods = 12,
).fit()
#get forecast per year
forecast_per_year = ts_model.forecast(steps = 30)

We calculate the residuals (differences between the fitted values and the actual data), compute the standard deviation of the residuals, and use this to calculate 95% confidence intervals for the forecasted values, based on a z-score of 1.96 for a 95% confidence level.

In [56]:
residuals = ts_model.resid
sigma = np.std(residuals) 
# Calculate the confidence intervals
alpha = 0.05 
z_score = 1.96 
# Calculate the lower and upper bounds of the confidence intervals
lower_bounds = forecast_per_year - z_score * sigma
upper_bounds = forecast_per_year + z_score * sigma

We create a DataFrame to store the forecasted smoke impact metric, its corresponding confidence intervals (lower and upper bounds), and the years. The lower confidence interval values are adjusted to ensure they do not go below zero, as negative values aren't possible for this metric.

In [57]:
forecast_df = pd.DataFrame({"Smoke_Impact_Metric" : list(forecast_per_year),
                            "Year" : pd.DataFrame(forecast_per_year).index,
                            "LowerCI" : list(lower_bounds),
                            "UpperCI" : list(upper_bounds)})
#Extract year
forecast_df.Year = forecast_df.Year.dt.year
#Shrink CI floor to 0, which is the lowest possible value
forecast_df.loc[forecast_df['LowerCI'] < 0, 'LowerCI'] = 0

The code extracts the year from the df_ts index, ensuring it is in datetime format and then converting it to just the year. After that, a new row is added to the df_ts DataFrame for the year 2021, using the forecasted smoke impact metric value from the forecast_df.

In [58]:
df_ts['Year'] = df_ts.index
df_ts['Year'] = pd.to_datetime(df_ts['Year'])
df_ts['Year'] = df_ts['Year'].dt.year
#add a row
df_ts = pd.concat([df_ts, 
                           pd.DataFrame({"Year" : [2021],
                                         "Smoke_Impact_Metric" : forecast_df[forecast_df["Year"] == 2021]["Smoke_Impact_Metric"]})])

This code generates an interactive Plotly plot displaying the observed smoke impact metric along with the forecasted values and their confidence intervals (CI). The plot includes:

- Observed smoke metric (blue line)
- Predicted smoke metric (red line)
- Lower confidence interval (light blue dashed line)
- Upper confidence interval (light blue shaded area)

In [59]:
# Create the figure
fig = go.Figure()

# Add the SmokeMetric as a line plot
fig.add_trace(go.Scatter(
    x=df_ts["Year"],
    y=df_ts['Smoke_Impact_Metric'],
    mode='lines+markers',
    name='Smoke Metric (Computed)',
    line=dict(color='blue')
))

#predicted smoke metric
fig.add_trace(go.Scatter(
    x=forecast_df["Year"],
    y=forecast_df['Smoke_Impact_Metric'],
    mode='lines+markers',
    name='Smoke Metric (Predicted)',
    line=dict(color='red')
))
#Lower CI
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['LowerCI'],
    mode='lines',
    name='Confidence Interval',
    line=dict(color='lightblue', dash='dash'),
    fill=None,
    showlegend = False
))
#Upper CI
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['UpperCI'],
    mode='lines',
    name='Confidence Interval',  # Same label for the upper bound
    line=dict(color='lightblue', dash='dash'),
    fill='tonexty',  # Fill the area between the lines
    fillcolor='rgba(173, 216, 230, 0.5)'  # Light blue fill
))

# Update layout
fig.update_layout(
    title='Smoke Metric Forecast Through 2050',
    xaxis_title='Year',
    yaxis_title='Smoke Metric',
    template='plotly_white',
    width = 1000,
    height = 750,
    font={
        'size': 25 
    },
    legend={
        'font': {
            'size' : 18
        }
    }
)

fig.write_image(os.path.join(image_dir, "forecast_smoke.png"))

fig.show()

This code prepares two dataframes (df_model_deaths and df_model_incidence) for polynomial feature transformation by dropping unnecessary columns and handling missing values. Then, it applies polynomial feature transformation with a degree of 3, excluding the bias term. The transformations are intended to create polynomial features for building models that predict deaths and incidence based on other available metrics.

In [60]:
poly = PolynomialFeatures(degree=3, include_bias=False)
df_model_deaths = df.drop(["Smoke_Impact_Metric_Old", "avg_AQI",  "Incidence"], axis = 1).dropna()
df_model_incidence = df.drop(["Smoke_Impact_Metric_Old", "avg_AQI", "Deaths"], axis = 1).dropna()

Next, we applied polynomial feature transformations to the data, creating additional columns such as "Rolling_Avg_Squared" and "Rolling_Avg_Cubed" based on the "Rolling_Avg" column. The resulting feature set is prepared for a regression analysis, with the target variable being "Incidence." We then fit a model using these features to explore the relationship between the rolling average of the smoke impact metric and the incidence of respiratory diseases.

In [61]:
X = df_model_incidence.drop(["Incidence", "Smoke_Impact_Metric", "EMA", "Year"], axis = 1)
X = pd.DataFrame(poly.fit_transform(X))
#rename columns
X.columns = ["Rolling_Avg", "Rolling_Avg_Squared", "Rolling_Avg_Cubed"]
X["Year"] = df_model_incidence["Year"]
X["Incidence"] = df_model_incidence["Incidence"]


In the following sections, we replicated the analysis for predicting both respiratory disease incidence and deaths using various forms of the smoke impact metric, including the moving average (Rolling_Avg), Exponentially Moving Average (EMA), and raw values. For each case, we first used an OLS regression model to predict the outcomes (incidence or deaths) based on the respective smoke impact metric features and the year.

We visualized the performance of the models by plotting predicted vs. true values of the outcome variable (incidence or deaths) using scatter plots with fitted OLS trendlines. These plots were created for each combination of smoke metric and outcome. In each case, the trendline helps us assess the linear relationship between the predicted and true values, indicating how well our models perform in capturing this relationship.

In [62]:
X = X.dropna()
Y = X["Incidence"]
X = X.drop(["Incidence"], axis = 1)
# Add a constant for the intercept term
X = sm.add_constant(X)

Y.index = X.index
# Fit the regression model
model_sm_incidence_rolling = sm.OLS(Y, X).fit()

# Get the summary
print(model_sm_incidence_rolling.summary())

                            OLS Regression Results                            
Dep. Variable:              Incidence   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     13.12
Date:                Mon, 02 Dec 2024   Prob (F-statistic):           2.10e-05
Time:                        11:30:45   Log-Likelihood:                -264.41
No. Observations:                  25   AIC:                             538.8
Df Residuals:                      20   BIC:                             544.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -4.089e+06   7

In [63]:
df_preds_sm_incidence = pd.DataFrame({"True Incidence" : Y,
                                        "Predicted Incidence": model_sm_incidence_rolling.predict()})

fig = px.scatter(x = df_preds_sm_incidence["True Incidence"],
           y = df_preds_sm_incidence["Predicted Incidence"],
           trendline = "ols")

fig.update_layout(
    xaxis_title = "True Incidence",
    yaxis_title = "Predicted Incidence",
    template = "plotly_white",
    title = "Predicting Incidence From Moving Avg Smoke Metric and Year",
    font={
        'size': 15
    }
)

In [64]:
X = df_model_deaths.drop(["Deaths", "Smoke_Impact_Metric", "EMA", "Year"], axis = 1)
X = pd.DataFrame(poly.fit_transform(X))
#rename columns
X.columns = ["Rolling_Avg", "Rolling_Avg_Squared", "Rolling_Avg_Cubed"]
X["Year"] = df_model_deaths["Year"]
X["Deaths"] = df_model_deaths["Deaths"]

In [65]:
X = X.dropna()
Y = X["Deaths"]
X = X.drop(["Deaths"], axis = 1)
# Add a constant for the intercept term
X = sm.add_constant(X)

Y.index = X.index
# Fit the regression model
model_sm_deaths_rolling = sm.OLS(Y, X).fit()

# Get the summary
print(model_sm_deaths_rolling.summary())

                            OLS Regression Results                            
Dep. Variable:                 Deaths   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     1326.
Date:                Mon, 02 Dec 2024   Prob (F-statistic):           4.75e-35
Time:                        11:30:46   Log-Likelihood:                -234.79
No. Observations:                  37   AIC:                             479.6
Df Residuals:                      32   BIC:                             487.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -3.271e+05   5

In [66]:
X = df_model_incidence.drop(["Incidence", "Rolling_Avg", "EMA", "Year"], axis = 1)
X = pd.DataFrame(poly.fit_transform(X))
#rename columns
X.columns = ["Smoke_Impact", "Smoke_Impact_Metric_Squared", "Smoke_Impact_Metric_Cubed"]
X["Year"] = df_model_incidence["Year"]
X["Incidence"] = df_model_incidence["Incidence"]

In [67]:
X = X.dropna()
Y = X["Incidence"]
X = X.drop(["Incidence"], axis = 1)
# Add a constant for the intercept term
X = sm.add_constant(X)

Y.index = X.index
# Fit the regression model
model_sm_incidence_metric = sm.OLS(Y, X).fit()

# Get the summary
print(model_sm_incidence_metric.summary())

                            OLS Regression Results                            
Dep. Variable:              Incidence   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     2.261
Date:                Mon, 02 Dec 2024   Prob (F-statistic):             0.0986
Time:                        11:30:46   Log-Likelihood:                -275.85
No. Observations:                  25   AIC:                             561.7
Df Residuals:                      20   BIC:                             567.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [68]:
X = df_model_deaths.drop(["Deaths", "Rolling_Avg", "EMA", "Year"], axis = 1)
X = pd.DataFrame(poly.fit_transform(X))
#rename columns
X.columns = ["Smoke_Impact", "Smoke_Impact_Metric_Squared", "Smoke_Impact_Metric_Cubed"]
X["Year"] = df_model_incidence["Year"]
X["Deaths"] = df_model_deaths["Deaths"]

In [69]:
X = X.dropna()
Y = X["Deaths"]
X = X.drop(["Deaths"], axis = 1)
# Add a constant for the intercept term
X = sm.add_constant(X)

Y.index = X.index
# Fit the regression model
model_sm_deaths_metric = sm.OLS(Y, X).fit()

# Get the summary
print(model_sm_deaths_metric.summary())

                            OLS Regression Results                            
Dep. Variable:                 Deaths   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     966.7
Date:                Mon, 02 Dec 2024   Prob (F-statistic):           7.32e-28
Time:                        11:30:46   Log-Likelihood:                -193.68
No. Observations:                  31   AIC:                             397.4
Df Residuals:                      26   BIC:                             404.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [70]:
X = df_model_incidence.drop(["Incidence", "Smoke_Impact_Metric", "Rolling_Avg", "Year"], axis = 1)
X = pd.DataFrame(poly.fit_transform(X))
#rename columns
X.columns = ["EMA_Avg", "EMA_Avg_Squared", "EMA_Avg_Cubed"]
X["Year"] = df_model_incidence["Year"]
X["Incidence"] = df_model_incidence["Incidence"]

In [71]:
X = X.dropna()
Y = X["Incidence"]
X = X.drop(["Incidence"], axis = 1)
# Add a constant for the intercept term
X = sm.add_constant(X)

Y.index = X.index
# Fit the regression model
model_sm_incidence_ema = sm.OLS(Y, X).fit()

# Get the summary
print(model_sm_incidence_ema.summary())

                            OLS Regression Results                            
Dep. Variable:              Incidence   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     12.18
Date:                Mon, 02 Dec 2024   Prob (F-statistic):           3.53e-05
Time:                        11:30:46   Log-Likelihood:                -265.08
No. Observations:                  25   AIC:                             540.2
Df Residuals:                      20   BIC:                             546.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -4.246e+06   7.78e+05     

### Forecast Disease Incidence & Deaths

 this section, we create a new dataframe df_forecast_health by removing the columns "Smoke_Impact_Metric_Old", "EMA", and "avg_AQI" from the original dataframe df. 

In [72]:
df_forecast_health = df.drop(["Smoke_Impact_Metric_Old", "EMA", "avg_AQI"], axis = 1)

We first filter the data for the years 2017 to 2020 and remove unnecessary columns. Next, we concatenate the filtered data with the existing forecast data and calculate the 8-year rolling average for the smoke impact metric. Finally, we clean up the dataset by removing any rows with missing values.

In [73]:
# Filter data for years 2017-2020 and drop unnecessary columns
subset_df = df_forecast_health[(df_forecast_health.Year >= 2013) & (df_forecast_health.Year <= 2020)].drop(["Rolling_Avg", "Deaths", "Incidence"], axis = 1)

# Concatenate the filtered subset with the existing forecast data and calculate the rolling average for 8 years
forecast_df = pd.concat([subset_df, forecast_df])
forecast_df['Rolling_Avg'] = forecast_df['Smoke_Impact_Metric'].rolling(window=8).mean()

# Remove rows with missing values from the forecast data
forecast_df = forecast_df.dropna()

We generate polynomial features from the rolling average of the forecast data, creating new columns for the squared and cubed terms. Next, we add the year column and include a constant for the intercept term. Finally, we prepare the dataset for modeling by adding the constant to the feature set.

In [74]:
# Generate polynomial features (squared and cubed terms) from the Rolling_Avg column
pred = pd.DataFrame(poly.fit_transform(pd.DataFrame(forecast_df['Rolling_Avg'])))
pred.columns = ["Rolling_Avg", "Rolling_Avg_Squared", "Rolling_Avg_Cubed"]  # Rename the columns for clarity

# Add the Year column to the polynomial features dataframe
pred['Year'] = forecast_df.Year

# Add a constant for the intercept term to the feature set
pred = sm.add_constant(pred)

With this code, we make predictions using the regression model, extract and clean up the output, and merge the results back into the forecast_df for further analysis.

In [75]:
# Get predictions from the regression model using the polynomial features (pred)
preds_incidence = model_sm_incidence_rolling.get_prediction(pred)

# Extract the summary frame of predictions, which includes confidence intervals
incidence_preds = preds_incidence.summary_frame()

# Drop unnecessary columns (confidence intervals for individual observations and standard error)
incidence_preds = incidence_preds.drop(["obs_ci_lower", "mean_se", "obs_ci_upper"], axis = 1)

# Rename the columns to better reflect the data
incidence_preds.columns = ["Incidence", "Incidence_CI_Lower", "Incidence_CI_Upper"]

# Merge the predicted incidence and confidence intervals into the forecast dataframe
forecast_df = pd.concat([forecast_df, incidence_preds], axis = 1)

Here we add year 2021 so the incidence is connected in the subsequent graph.

In [76]:
df_forecast_health = pd.concat([df_forecast_health, forecast_df[forecast_df.Year == 2021]])

### Plot

We next create a plot with two line traces:

- Computed Smoke Metric (Rolling Average): Displays the moving average of the smoke impact metric (as calculated earlier), shown in blue.
- Predicted Smoke Metric (Rolling Average): Displays the predicted values for the rolling average of the smoke impact metric, shown in red.

In [77]:
# Create the figure
fig = go.Figure()

# Add the SmokeMetric as a line plot
fig.add_trace(go.Scatter(
    x=df_forecast_health["Year"],
    y=df_forecast_health['Rolling_Avg'],
    mode='lines+markers',
    name='Incidence (Computed)',
    line=dict(color='blue')
))

#predicted smoke metric
fig.add_trace(go.Scatter(
    x=forecast_df["Year"],
    y=forecast_df['Rolling_Avg'],
    mode='lines+markers',
    name='Incidence (Predicted)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title='Smoke Metric Rolling Average (Computed and Predicted)',
    xaxis_title='Year',
    yaxis_title='Moving Average Smoke Metric',
    template='plotly_white',
    width = 750,
    height = 500
)

fig.show()

We next create a plot with two line traces:

- Computed Incidence (Rolling Average): Displays the incidence of chronic respiratory disease measured thus far. 
- Predicted Incidence (Rolling Average): Displays the predicted values for chronic respiratory disease. 

In [78]:
# Create the figure
fig1 = go.Figure()

# Add the SmokeMetric as a line plot
fig1.add_trace(go.Scatter(
    x=df_forecast_health["Year"],
    y=df_forecast_health['Incidence'],
    mode='lines+markers',
    name='Incidence (Computed)',
    line=dict(color='blue')
))

#predicted smoke metric
fig1.add_trace(go.Scatter(
    x=forecast_df["Year"],
    y=forecast_df['Incidence'],
    mode='lines+markers',
    name='Incidence (Predicted)',
    line=dict(color='red')
))

#Lower CI
fig1.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Incidence_CI_Lower'],
    mode='lines',
    name='Confidence Interval',
    line=dict(color='lightblue', dash='dash'),
    fill=None,
    showlegend = False
))
#Upper CI
fig1.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Incidence_CI_Upper'],
    mode='lines',
    name='Confidence Interval',  # Same label for the upper bound
    line=dict(color='lightblue', dash='dash'),
    fill='tonexty',  # Fill the area between the lines
    fillcolor='rgba(173, 216, 230, 0.5)'  # Light blue fill
))

# Update layout
fig1.update_layout(
    title='Incidence (Computed and Predicted)',
    xaxis_title='Year',
    yaxis_title='Incidence',
    template='plotly_white',
    width = 750,
    height = 500
)

fig1.show()